# Km3Kit : Overview



## Setup

In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
from matplotlib import MatplotlibDeprecationWarning

# this in general not advised, but for this specific notebook
# I'd like to ignore some deprecation warnings by matplotlib
import warnings
warnings.filterwarnings(
    "ignore", category=MatplotlibDeprecationWarning
)

In [2]:
import astropy.units as u
from astropy.coordinates import SkyCoord

In [3]:
from gammapy.data import EventList

In [10]:
#events_Arca21 = EventList.read("data/fermi-3fhl-gc-events.fits")
events_Arca21 = EventList.read("../data/arca21_bdt_converted_Data.fits")

In [11]:
events_Arca21.table

RUN_ID,EVENT_ID,TYPE,RA,DEC,ENERGY,TIME
,,,deg,deg,GeV,s
float64,float64,float64,float32,float32,float32,float64
13288.0,1.0,0.0,-57238.484,-57238.484,0.0,59844.6268298612
13288.0,2.0,0.0,-57238.484,-57238.484,0.0,59844.62684143521
13288.0,3.0,0.0,-57238.484,-57238.484,0.0,59844.62685879646
13288.0,4.0,4000.0,256.91464,3.3188646,1491.3102,59844.62685995363
13288.0,5.0,0.0,-57238.484,-57238.484,0.0,59844.626865740865
13288.0,6.0,4000.0,272.96637,-0.6672938,0.02423879,59844.626866898034
13288.0,7.0,0.0,-57238.484,-57238.484,0.0,59844.62687152764
13288.0,8.0,4000.0,219.23274,61.264732,337.3898,59844.62687384244


You can do *len* over event_Arca21.table to find the total number of events.

In [12]:
len(events_Arca21.table)

49857399

And we can access any other attribute of the `Table` object as well:

In [13]:
events_Arca21.table.colnames

['RUN_ID', 'EVENT_ID', 'TYPE', 'RA', 'DEC', 'ENERGY', 'TIME']

For convenience we can access the most important event parameters as properties on the `EventList` objects. The attributes will return corresponding Astropy objects to represent the data, such as [astropy.units.Quantity](http://docs.astropy.org/en/stable/api/astropy.units.Quantity.html), [astropy.coordinates.SkyCoord](http://docs.astropy.org/en/stable/api/astropy.coordinates.SkyCoord.html) or [astropy.time.Time](http://docs.astropy.org/en/stable/api/astropy.time.Time.html#astropy.time.Time) objects:

In [14]:
events_Arca21.energy.to("GeV")

<Quantity [0.0000000e+00, 0.0000000e+00, 0.0000000e+00, ..., 3.2421611e+03,
           3.8405530e-02, 0.0000000e+00] GeV>

In [15]:
events_Arca21.galactic

ValueError: Latitude angle(s) must be within -90 deg <= angle <= 90 deg, got [-5.7238484e+04 -5.7238484e+04 -5.7238484e+04 ...  3.9285011e+01
 -3.4922657e+00 -5.7238484e+04] deg

In [16]:
events_Arca21.time

<Time object: scale='tt' format='mjd' value=[0.69264614 0.69264614 0.69264614 ... 0.69366898 0.69366898 0.69366898]>

There is also some convenience to plot the events:

In [17]:
events_Arca21.plot_image()

ValueError: Latitude angle(s) must be within -90 deg <= angle <= 90 deg, got [-5.7238484e+04 -5.7238484e+04 -5.7238484e+04 ...  3.9285011e+01
 -3.4922657e+00 -5.7238484e+04] deg

In addition `EventList` provides convenience methods to filter the event lists. One possible use case is to find the highest energy event within a radius of 0.5 deg around the vela position:

In [ ]:
# select all events within a radius of 0.5 deg around center
from regions import CircleSkyRegion

center = SkyCoord("0d", "0d", frame="galactic")
region = CircleSkyRegion(center, radius=0.5 * u.deg)
events_gc_Arca21 = events_Arca21.select_region(region)

# sort events by energy
events_gc_Arca21.table.sort("ENERGY")

# and show highest energy photon
events_gc_Arca21.energy[-1].to("GeV")

## 2. Maps

The [gammapy.maps]() sub package contains classes to work with sky images and cubes.

In [ ]:
from gammapy.maps import Map

gc_Arca21 = Map.create(
    width=(20 * u.deg, 10 * u.deg),
    skydir=center,
    proj="CAR",
    binsz=0.05 *u.deg,
    map_type="wcs",
    frame="galactic"
)

The image is a `~gammapy.maps.WcsNDMap` object:

In [ ]:
gc_Arca21

The shape of the image is 400 x 200 pixel and it is defined using a cartesian projection in galactic coordinates.

The ``geom`` attribute is a `~gammapy.maps.WcsGeom` object:

In [ ]:
print(gc_Arca21.geom)

Now we can fill the events in the map and plot it:

In [ ]:
gc_Arca21.fill_events(events_Arca21)
gc_Arca21.plot(stretch="sqrt", cmap="inferno");

The ``plot`` method basically calls [plt.imshow](http://matplotlib.org/api/pyplot_api.html#matplotlib.pyplot.imshow), passing the `gc_Arca21.data` attribute but in addition handles axis with world coordinates using [astropy.visualization.wcsaxes](https://docs.astropy.org/en/stable/visualization/wcsaxes/) and defines some defaults for nicer plots (e.g. the colormap 'afmhot').

Now we can also take a look at the `.data` attribute: 

In [ ]:
gc_Arca21.data

That looks familiar! It just an *ordinary* 2 dimensional numpy array,  which means you can apply any known numpy method to it:

In [ ]:
print(f"Total number of counts in the image: {gc_Arca21.data.sum():.0f}")

In [ ]:
from gammapy.maps import MapAxis

In [ ]:
energy_axis = MapAxis.from_energy_bounds(
    energy_min="10 GeV", energy_max="2 TeV", nbin=5
)

In [ ]:
print(energy_axis)

In [ ]:
gc_Arca21_cube = Map.create(
    width=(20 * u.deg, 10 * u.deg),
    skydir=center,
    proj="CAR",
    binsz=0.05 *u.deg,
    map_type="wcs",
    frame="galactic",
    axes=[energy_axis]
)

In [ ]:
print(gc_Arca21_cube)

In [ ]:
gc_Arca21_cube.fill_events(events_Arca21)

To make the structures in the image more visible we will smooth the data using a Gaussian kernel.

In [ ]:
gc_Arca21_cube_smoothed = gc_Arca21_cube.smooth(
    kernel="gauss", width=0.1 * u.deg
)

To visualise the data cube we can use interactive plotting:

In [ ]:
gc_Arca21_cube_smoothed.plot_interactive(cmap="inferno")

Or plot the image in energy bands as a grid:

In [60]:
gc_Arca21_cube_smoothed.plot_grid(
    ncols=3, figsize=(12, 5), cmap="inferno", stretch="sqrt"
);

You can also do a rectangular cutout of a certain region in the image:

In [61]:
# define center and size of the cutout region
center = SkyCoord(0, 0, unit="deg", frame="galactic")
gc_Arca21_cutout = gc_Arca21_cube_smoothed.cutout(center, 9 * u.deg)
gc_Arca21_cutout.plot_interactive(stretch="sqrt", cmap="inferno");

interactive(children=(SelectionSlider(continuous_update=False, description='Select energy:', layout=Layout(wid…

For a more detailed introduction to `gammapy.maps`, take a look a the [maps.ipynb](https://docs.gammapy.org/0.18.2/tutorials/maps.html) notebook.

## 3. Source catalogs

Gammapy provides a convenient interface to access and work with catalog based data. 

Let's start with importing the 3FHL catalog object from the `~gammapy.catalog` submodule:

In [62]:
from gammapy.catalog import SourceCatalog3FHL

First we initialize the Fermi-LAT 3FHL catalog and directly take a look at the `.table` attribute:

In [63]:
fermi_Arca21 = SourceCatalog3FHL("data/gll_psch_v13.fit.gz")
fermi_Arca21.table

Source_Name,RAJ2000,DEJ2000,GLON,GLAT,Conf_95_SemiMajor,Conf_95_SemiMinor,Conf_95_PosAng,ROI_num,Signif_Avg,Pivot_Energy,Flux_Density,Unc_Flux_Density,Flux,Unc_Flux,Energy_Flux,Unc_Energy_Flux,Signif_Curve,SpectrumType,Spectral_Index,Unc_Spectral_Index,beta,Unc_beta,PowerLaw_Index,Unc_PowerLaw_Index,Flux_Band,Unc_Flux_Band,nuFnu,Sqrt_TS_Band,Npred,HEP_Energy,HEP_Prob,Variability_BayesBlocks,Extended_Source_Name,ASSOC_GAM,TEVCAT_FLAG,ASSOC_TEV,CLASS,ASSOC1,ASSOC2,ASSOC_PROB_BAY,ASSOC_PROB_LR,Redshift,NuPeak_obs
,deg,deg,deg,deg,deg,deg,deg,,,GeV,1 / (GeV s cm2),1 / (GeV s cm2),1 / (s cm2),1 / (s cm2),erg / (s cm2),erg / (s cm2),,,,,,,,,1 / (s cm2),1 / (s cm2),erg / (s cm2),,,GeV,,,,,,,,,,,,,Hz
bytes18,float32,float32,float32,float32,float32,float32,float32,int16,float32,float32,float32,float32,float32,float32,float32,float32,float32,bytes11,float32,float32,float32,float32,float32,float32,float32[5],"float32[5,2]",float32[5],float32[5],float32,float32,float32,int16,bytes18,bytes18,bytes1,bytes21,bytes7,bytes26,bytes26,float32,float32,float32,float32
3FHL J0001.2-0748,0.3107,-7.8075,89.0094,-67.3118,0.0424,0.0424,--,64,5.362,23.73,5.3174e-13,2.0975e-13,2.9593e-11,1.1704e-11,1.6752e-12,1.0743e-12,1.02,PowerLaw,1.6724,0.8274,0.5916,0.7129,2.2226,0.4808,1.1127661e-11 .. 1.1422301e-22,-6.0763976e-12 .. 6.529277e-12,3.533989e-13 .. 1.1789072e-22,3.1458344 .. 0.0,7.63,86.975,0.9964,1,,3FGL J0001.2-0748,N,,bll,PMN J0001-0746,,0.9974,0.9721,--,306196370000000.0
3FHL J0001.9-4155,0.4849,-41.9303,334.1216,-72.0697,0.1018,0.1018,--,429,5.638,28.42,5.4253e-13,1.6839e-13,4.3230e-11,1.3428e-11,3.4900e-12,1.8276e-12,0.45,PowerLaw,1.7819,0.4941,0.1187,0.2798,1.9418,0.3100,2.1003905e-11 .. 1.9287885e-18,-8.032091e-12 .. 5.8594097e-12,6.7452245e-13 .. 2.078675e-18,4.899907 .. 0.0,12.51,266.625,0.9622,1,,3FGL J0002.2-4152,N,,bcu,1RXS J000135.5-415519,,0.9960,0.0000,--,6309576500000000.0
3FHL J0002.1-6728,0.5283,-67.4825,310.0868,-48.9549,0.0357,0.0357,--,386,8.470,20.82,1.2062e-12,3.2106e-13,5.0093e-11,1.3349e-11,2.3058e-12,9.5580e-13,1.53,PowerLaw,1.8109,0.6260,0.7933,0.5956,2.4285,0.3710,2.4550664e-11 .. 1.9009976e-21,-8.634195e-12 .. 4.8021903e-12,7.7340695e-13 .. 1.9026535e-21,5.900217 .. 0.0,17.11,52.152,0.9988,1,,3FGL J0002.0-6722,N,,bcu,SUMSS J000215-672653,,0.0000,0.9395,--,4466832000000000.0
3FHL J0003.3-5248,0.8300,-52.8150,318.9245,-62.7936,0.0425,0.0425,--,145,7.229,23.66,7.5065e-13,2.3102e-13,4.1560e-11,1.2839e-11,2.2874e-12,1.1145e-12,1.70,PowerLaw,1.6010,0.5644,0.9972,0.1721,2.2481,0.3732,2.0886386e-11 .. 7.5867555e-23,-8.143967e-12 .. 5.31299e-12,6.6265456e-13 .. 7.800202e-23,5.298393 .. 0.0,13.02,67.310,0.9636,1,,3FGL J0003.2-5246,N,,bcu,RBS 0006,,0.9996,0.9716,--,7.079464e+16
3FHL J0007.0+7303,1.7647,73.0560,119.6625,10.4666,0.0101,0.0101,--,277,75.265,12.80,1.7436e-10,7.5950e-12,1.5308e-09,6.1341e-11,3.6785e-11,1.5973e-12,3.24,LogParabola,3.1751,0.2103,0.9021,0.2659,3.8315,0.1141,1.3514667e-09 .. 3.839895e-18,-5.7581186e-11 .. 4.060418e-12,4.109739e-11 .. 2.9231144e-18,71.33829 .. 0.0,654.15,60.292,0.9972,1,,3FGL J0007.0+7302,E,CTA 1,PSR,LAT PSR J0007+7303,,1.0000,0.0000,--,--
3FHL J0007.9+4711,1.9931,47.1920,115.3093,-15.0354,0.0196,0.0196,--,302,17.774,17.19,5.9778e-12,8.7683e-13,1.5131e-10,2.2181e-11,5.1444e-12,1.0540e-12,0.56,PowerLaw,2.6783,0.4196,0.1696,0.3282,2.8588,0.2685,1.0582407e-10 .. 1.9819723e-16,-1.7538379e-11 .. 4.823511e-12,3.278615e-12 .. 1.8668298e-16,15.209969 .. 0.0,50.95,68.152,0.9759,1,,3FGL J0008.0+4713,N,,bll,MG4 J000800+4712,,1.0000,0.9873,0.2800,2511884200000000.0
3FHL J0008.4-2339,2.1243,-23.6514,50.2908,-79.7021,0.0366,0.0366,--,517,9.679,16.96,3.0610e-12,7.3475e-13,7.4602e-11,1.7896e-11,2.4733e-12,8.1716e-13,0.34,PowerLaw,2.7388,0.7145,0.1737,0.5618,2.9070,0.4520,5.804992e-11 .. 1.1117311e-20,-1.4419374e-11 .. 6.10661e-12,1.7951775e-12 .. 1.0403958e-20,9.133706 .. 0.0,19.83,71.122,0.9968,1,,3FGL J0008.6-2340,N,,bll,RBS 0016,,0.9996,0.9673,0.1470,524807800000000.0
3FHL J0009.1+0628,2.2874,6.4814,104.4

This looks very familiar again. The data is just stored as an [astropy.table.Table](http://docs.astropy.org/en/stable/api/astropy.table.Table.html#astropy.table.Table) object. We have all the methods and attributes of the `Table` object available. E.g. we can sort the underlying table by `Signif_Avg` to find the top 5 most significant sources:



In [64]:
# sort table by significance
fermi_Arca21.table.sort("Signif_Avg")

# invert the order to find the highest values and take the top 5
top_five_TS_Arca21 = fermi_Arca21.table[::-1][:5]

# print the top five significant sources with association and source class
top_five_TS_Arca21[["Source_Name", "ASSOC1", "ASSOC2", "CLASS", "Signif_Avg"]]

Source_Name,ASSOC1,ASSOC2,CLASS,Signif_Avg
bytes18,bytes26,bytes26,bytes7,float32
3FHL J0534.5+2201,Crab Nebula,,PWN,168.641
3FHL J1104.4+3812,Mkn 421,,BLL,144.406
3FHL J0835.3-4510,PSR J0835-4510,Vela X field,PSR,138.801
3FHL J0633.9+1746,PSR J0633+1746,,PSR,99.734
3FHL J1555.7+1111,PG 1553+113,,BLL,94.411


If you are interested in the data of an individual source you can access the information from catalog using the name of the source or any alias source name that is defined in the catalog:

In [65]:
mkn_421_Arca21 = fermi_Arca21["3FHL J1104.4+3812"]

# or use any alias source name that is defined in the catalog
mkn_421_Arca21 = fermi_Arca21["Mkn 421"]
print(mkn_421_Arca21.data["Signif_Avg"])

144.40611


Now we can plot the sources on the image created above:

In [66]:
_, ax, _ = gc_Arca21.smooth("0.1 deg").plot(
    stretch="sqrt", cmap="inferno"
);

positions = fermi_Arca21.positions
ax.scatter(
    positions.data.lon.deg,
    positions.data.lat.deg,
    transform=ax.get_transform("icrs"),
    color="w",
    marker="x"
)

TypeError: cannot unpack non-iterable WCSAxes object

## 4. Spectral Models and Flux points

In the previous section we learned how access basic data from individual sources in the catalog. Now we will go one step further and explore the full spectral information of sources.

As a first example we will start with the Crab Nebula:

In [ ]:
crab_Arca21 = fermi_Arca21["Crab Nebula"]
crab_Arca21_model = crab_Arca21.sky_model()
print(crab_Arca21_model)

In [ ]:
crab_Arca21_spec = crab_Arca21_model.spectral_model

NameError: name 'crab_3fhl_model' is not defined

The `crab_Arca21.spectral_model` is an instance of the `PowerLaw2SpectralModel` model, with the parameter values and errors taken from the 3FHL catalog. 

Let's plot the spectral model in the energy range between 10 GeV and 2000 GeV:

In [ ]:
ax_crab_Arca21 = crab_Arca21_spec.plot(
    energy_range=[10, 2000] * u.GeV,
)

NameError: name 'crab_3fhl_spec' is not defined

We assign the return axes object to variable called `ax_crab_Arca21`, because we will re-use it later to plot the flux points on top.

To compute the differential flux at 100 GeV we can simply call the model like normal Python function and convert to the desired units:

In [ ]:
crab_Arca21_spec(100 * u.GeV).to("cm-2 s-1 GeV-1")

Next we can compute the integral flux of the Crab between 10 GeV and 2000 GeV:

In [ ]:
crab_Arca21_spec.integral(
    energy_min=10 * u.GeV, energy_max=2 * u.TeV
).to("cm-2 s-1")

We can easily convince ourself, that it corresponds to the value given in the Fermi-LAT 3FHL catalog:

In [ ]:
crab_Arca21.data["Flux"]

In addition we can compute the energy flux between 10 GeV and 2000 GeV:

In [ ]:
crab_Arca21_spec.energy_flux(
    energy_min=10 * u.GeV, energy_max=2000 * u.GeV
).to("erg cm-2 s-1")

Next we will access the flux points data of the Crab:

In [ ]:
print(crab_Arca21.flux_points)

If you want to learn more about the different flux point formats you can read the specification [here](https://gamma-astro-data-formats.readthedocs.io/en/latest/spectra/flux_points/index.html).

No we can check again the underlying astropy data structure by accessing the `.table` attribute:

In [ ]:
crab_Arca21.flux_points.table

Finally let's combine spectral model and flux points in a single plot and scale with `energy_power=2` to obtain the spectral energy distribution:

In [ ]:
ax = crab_Arca21_spec.plot(
    energy_range=[10, 2000] * u.GeV, energy_power=2
)

ax = crab_Arca21_spec.plot_error(
    energy_range=[10, 2000] * u.GeV,
    energy_power=2,
    facecolor="tab:blue"
)

fp = crab_Arca21.flux_points.to_sed_type("dnde")
fp.plot(ax=ax, energy_power=2);